In [1]:
import torch
import torch.nn as nn

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_op = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv_op(x)


class DownSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = DoubleConv(in_channels, out_channels)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        down = self.conv(x)
        p = self.pool(down)

        return down, p


class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels//2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
       x1 = self.up(x1)
       x = torch.cat([x1, x2], 1)
       return self.conv(x)
    
class UNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.down_convolution_1 = DownSample(in_channels, 64)
        self.down_convolution_2 = DownSample(64, 128)
        self.down_convolution_3 = DownSample(128, 256)
        self.down_convolution_4 = DownSample(256, 512)

        self.bottle_neck = DoubleConv(512, 1024)

        self.up_convolution_1 = UpSample(1024, 512)
        self.up_convolution_2 = UpSample(512, 256)
        self.up_convolution_3 = UpSample(256, 128)
        self.up_convolution_4 = UpSample(128, 64)
        
        self.out = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=1)

    def forward(self, x):
       down_1, p1 = self.down_convolution_1(x)
       down_2, p2 = self.down_convolution_2(p1)
       down_3, p3 = self.down_convolution_3(p2)
       down_4, p4 = self.down_convolution_4(p3)

       b = self.bottle_neck(p4)

       up_1 = self.up_convolution_1(b, down_4)
       up_2 = self.up_convolution_2(up_1, down_3)
       up_3 = self.up_convolution_3(up_2, down_2)
       up_4 = self.up_convolution_4(up_3, down_1)

       out = self.out(up_4)
       return out

import os
from PIL import Image
from torch.utils.data.dataset import Dataset
from torchvision import transforms

class CMB_Dataset(Dataset):
    def __init__(self, root_path, test=False):
        self.root_path = root_path
        if test:
            self.images = sorted([root_path+"/test_data/"+i for i in os.listdir(root_path+"/test_data/")])
            self.masks = sorted([root_path+"/test_masks/"+i for i in os.listdir(root_path+"/test_masks/")])
        else:
            self.images = sorted([root_path+"/train_data/"+i for i in os.listdir(root_path+"/train_data/")])
            self.masks = sorted([root_path+"/train_masks/"+i for i in os.listdir(root_path+"/train_masks/")])
        
        self.transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor()])

    def __getitem__(self, index):
        img = Image.open(self.images[index]).convert("RGB")
        mask = Image.open(self.masks[index]).convert("L")

        return self.transform(img), self.transform(mask)

    def __len__(self):
        return len(self.images)    

import torch
from torch import optim, nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm


LEARNING_RATE = 3e-4
BATCH_SIZE = 32
EPOCHS = 2
DATA_PATH = "/data"
MODEL_SAVE_PATH = "/models/unet.pth"

device = "cuda" if torch.cuda.is_available() else "cpu"
train_dataset = CMB_Dataset(DATA_PATH)

generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(train_dataset, [0.8, 0.2], generator=generator)

train_dataloader = DataLoader(dataset=train_dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=True)

model = UNet(in_channels=3, num_classes=1).to(device)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(EPOCHS)):
    model.train()
    train_running_loss = 0
    for idx, img_mask in enumerate(tqdm(train_dataloader)):
        img = img_mask[0].float().to(device)
        mask = img_mask[1].float().to(device)

        y_pred = model(img)
        optimizer.zero_grad()

        loss = criterion(y_pred, mask)
        train_running_loss += loss.item()
        
        loss.backward()
        optimizer.step()

    train_loss = train_running_loss / (idx + 1)

    model.eval()
    val_running_loss = 0
    with torch.no_grad():
        for idx, img_mask in enumerate(tqdm(val_dataloader)):
            img = img_mask[0].float().to(device)
            mask = img_mask[1].float().to(device)
            
            y_pred = model(img)
            loss = criterion(y_pred, mask)

            val_running_loss += loss.item()

        val_loss = val_running_loss / (idx + 1)

    print("-"*30)
    print(f"Train Loss EPOCH {epoch+1}: {train_loss:.4f}")
    print(f"Valid Loss EPOCH {epoch+1}: {val_loss:.4f}")
    print("-"*30)

torch.save(model.state_dict(), MODEL_SAVE_PATH)


In [1]:
import os
import numpy as np
from torch.utils.data import Dataset
import torch

class CMB_Dataset(Dataset):
    def __init__(self, root_path, test=False):
        self.root_path = root_path
        if test:
            self.images = sorted([os.path.join(root_path, "test_data", i) for i in os.listdir(os.path.join(root_path, "test_data"))])
            self.masks = sorted([os.path.join(root_path, "test_masks", i) for i in os.listdir(os.path.join(root_path, "test_masks"))])
        else:
            self.images = sorted([os.path.join(root_path, "train_data", i) for i in os.listdir(os.path.join(root_path, "train_data"))])
            self.masks = sorted([os.path.join(root_path, "train_masks", i) for i in os.listdir(os.path.join(root_path, "train_masks"))])

    def __getitem__(self, index):
        ilc_cmb = np.load(self.images[index])
        true_cmb = np.load(self.masks[index])

        return torch.tensor(ilc_cmb, dtype=torch.float32), torch.tensor(true_cmb, dtype=torch.float32)

    def __len__(self):
        return len(self.images)

# Example usage:
DATA_PATH = "data"
train_dataset = CMB_Dataset(DATA_PATH)


0

In [43]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import os
# Define the UNet Model
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_op = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv_op(x)

class DownSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = DoubleConv(in_channels, out_channels)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        down = self.conv(x)
        p = self.pool(down)
        return down, p

# class UpSample(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super().__init__()
#         self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
#         self.conv = DoubleConv(in_channels, out_channels)

#     def forward(self, x1, x2):
#         x1 = self.up(x1)
#         x = torch.cat([x1, x2], dim=1)
#         return self.conv(x)

import torch
import torch.nn as nn

# class UpSample(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super().__init__()
#         self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
#         self.conv = DoubleConv(in_channels, out_channels)

#     def forward(self, x1, x2):
#         x1 = self.up(x1)
#         # Crop the feature map from the encoder to match the size of the upsampled feature map
#         diffY = x2.size()[2] - x1.size()[2]
#         diffX = x2.size()[3] - x1.size()[3]

#         x2 = x2[:, :, diffY // 2: diffY // 2 + x1.size()[2], diffX // 2: diffX // 2 + x1.size()[3]]

#         x = torch.cat([x1, x2], dim=1)
#         return self.conv(x)

class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        # Upsample x1
        x1 = self.up(x1)

        # Calculate the differences in dimensions
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        # Correct x1 dimensions to match x2's dimensions exactly
        if diffY > 0:
            # Pad if x1 is smaller than x2
            x1 = F.pad(x1, (0, 0, diffY // 2, diffY - diffY // 2))
        elif diffY < 0:
            # Crop if x1 is larger than x2
            x1 = x1[:, :, -diffY // 2: x1.size()[2] + diffY // 2, :]

        if diffX > 0:
            x1 = F.pad(x1, (diffX // 2, diffX - diffX // 2, 0, 0))
        elif diffX < 0:
            x1 = x1[:, :, :, -diffX // 2: x1.size()[3] + diffX // 2]

        # Concatenate the feature maps
        x = torch.cat([x1, x2], dim=1)
        return self.conv(x)


import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.down_convolution_1 = DownSample(in_channels, 64)
        self.down_convolution_2 = DownSample(64, 128)
        self.down_convolution_3 = DownSample(128, 256)
        self.down_convolution_4 = DownSample(256, 512)

        self.bottle_neck = DoubleConv(512, 1024)

        self.up_convolution_1 = UpSample(1024, 512)
        self.up_convolution_2 = UpSample(512, 256)
        self.up_convolution_3 = UpSample(256, 128)
        self.up_convolution_4 = UpSample(128, 64)
        
        self.out = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=1)

    def forward(self, x):
        down_1, p1 = self.down_convolution_1(x)
        down_2, p2 = self.down_convolution_2(p1)
        down_3, p3 = self.down_convolution_3(p2)
        down_4, p4 = self.down_convolution_4(p3)

        b = self.bottle_neck(p4)

        up_1 = self.up_convolution_1(b, down_4)
        up_2 = self.up_convolution_2(up_1, down_3)
        up_3 = self.up_convolution_3(up_2, down_2)
        up_4 = self.up_convolution_4(up_3, down_1)

        out = self.out(up_4)
        
        # Crop the output to match the input size
        diffY = out.size()[2] - x.size()[2]
        diffX = out.size()[3] - x.size()[3]

        if diffY != 0 or diffX != 0:
            out = out[:, :, diffY // 2: out.size()[2] - diffY // 2, diffX // 2: out.size()[3] - diffX // 2]
        
        return out



# class UNet(nn.Module):
#     def __init__(self, in_channels, num_classes):
#         super().__init__()
#         self.down_convolution_1 = DownSample(in_channels, 64)
#         self.down_convolution_2 = DownSample(64, 128)
#         self.down_convolution_3 = DownSample(128, 256)
#         self.down_convolution_4 = DownSample(256, 512)

#         self.bottle_neck = DoubleConv(512, 1024)

#         self.up_convolution_1 = UpSample(1024, 512)
#         self.up_convolution_2 = UpSample(512, 256)
#         self.up_convolution_3 = UpSample(256, 128)
#         self.up_convolution_4 = UpSample(128, 64)
        
#         self.out = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=1)

#     def forward(self, x):
#         down_1, p1 = self.down_convolution_1(x)
#         down_2, p2 = self.down_convolution_2(p1)
#         down_3, p3 = self.down_convolution_3(p2)
#         down_4, p4 = self.down_convolution_4(p3)

#         b = self.bottle_neck(p4)

#         up_1 = self.up_convolution_1(b, down_4)
#         up_2 = self.up_convolution_2(up_1, down_3)
#         up_3 = self.up_convolution_3(up_2, down_2)
#         up_4 = self.up_convolution_4(up_3, down_1)

#         out = self.out(up_4)
#         return out

from torchvision.transforms import functional as F
from PIL import Image

# Define the Dataset for .npy files
class CMB_Dataset(Dataset):
    def __init__(self, root_path, test=False, target_size=(512, 512)):
        self.root_path = root_path
        self.target_size = target_size
        if test:
            self.images = sorted([root_path+"/test_data/"+i for i in os.listdir(root_path+"/test_data/")])
            self.masks = sorted([root_path+"/test_masks/"+i for i in os.listdir(root_path+"/test_masks/")])
        else:
            self.images = sorted([root_path+"/train_data/"+i for i in os.listdir(root_path+"/train_data/")])
            self.masks = sorted([root_path+"/train_masks/"+i for i in os.listdir(root_path+"/train_masks/")])

    def __getitem__(self, index):
        img = np.real(np.load(self.images[index]))
        mask = np.real(np.load(self.masks[index]))

        # Convert numpy arrays to PIL images for resizing
        img_pil = Image.fromarray(img)
        mask_pil = Image.fromarray(mask)

        # Resize the images
        img_resized = F.resize(img_pil, self.target_size)
        mask_resized = F.resize(mask_pil, self.target_size)

        # Convert back to tensor and add channel dimension
        img_tensor = F.to_tensor(img_resized)
        mask_tensor = F.to_tensor(mask_resized)

        return img_tensor, mask_tensor
    # def __init__(self, root_path, test=False):
    #     self.root_path = root_path
    #     if test:
    #         self.images = sorted([root_path+"/test_data/"+i for i in os.listdir(root_path+"/test_data/")])
    #         self.masks = sorted([root_path+"/test_masks/"+i for i in os.listdir(root_path+"/test_masks/")])
    #     else:
    #         self.images = sorted([root_path+"/train_data/"+i for i in os.listdir(root_path+"/train_data/")])
    #         self.masks = sorted([root_path+"/train_masks/"+i for i in os.listdir(root_path+"/train_masks/")])
        
    #     self.transform = transforms.Compose([
    #         transforms.Resize((512, 512)),
    #         transforms.ToTensor()])

    # def __getitem__(self, index):
    #     img = np.load(self.images[index])
    #     mask = np.load(self.masks[index])

    #     img = img[np.newaxis, :, :]  # Add channel dimension
    #     mask = mask[np.newaxis, :, :]  # Add channel dimension

    #     return torch.tensor(img, dtype=torch.float32), torch.tensor(mask, dtype=torch.float32)

    def __len__(self):
        return len(self.images)    

# Training code
import torch
from torch import optim, nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

LEARNING_RATE = 3e-4
BATCH_SIZE = 32
EPOCHS = 10
DATA_PATH = "data"
MODEL_SAVE_PATH = "models/unet.pth"

device = "cuda" if torch.cuda.is_available() else "cpu"
train_dataset = CMB_Dataset(DATA_PATH)

generator = torch.Generator().manual_seed(1)
train_dataset, val_dataset = random_split(train_dataset, [0.8, 0.2], generator=generator)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=True)

model = UNet(in_channels=1, num_classes=1).to(device)  # Adjusted input channels
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()  # MSELoss is more appropriate for residual prediction

for epoch in tqdm(range(EPOCHS)):
    model.train()
    train_running_loss = 0
    for idx, img_mask in enumerate(tqdm(train_dataloader)):
        img = img_mask[0].to(device)
        mask = img_mask[1].to(device)

        y_pred = model(img)
        optimizer.zero_grad()

        loss = criterion(y_pred, mask)
        train_running_loss += loss.item()
        
        loss.backward()
        optimizer.step()

    train_loss = train_running_loss / (idx + 1)

    model.eval()
    val_running_loss = 0
    with torch.no_grad():
        for idx, img_mask in enumerate(tqdm(val_dataloader)):
            img = img_mask[0].to(device)
            mask = img_mask[1].to(device)
            
            y_pred = model(img)
            loss = criterion(y_pred, mask)

            val_running_loss += loss.item()

        val_loss = val_running_loss / (idx + 1)

    print("-"*30)
    print(f"Train Loss EPOCH {epoch+1}: {train_loss:.4f}")
    print(f"Valid Loss EPOCH {epoch+1}: {val_loss:.4f}")
    print("-"*30)

torch.save(model.state_dict(), MODEL_SAVE_PATH)


 10%|█         | 1/10 [00:00<00:03,  2.48it/s]

------------------------------
Train Loss EPOCH 1: 0.0049
Valid Loss EPOCH 1: 0.0025
------------------------------


 20%|██        | 2/10 [00:00<00:03,  2.50it/s]

------------------------------
Train Loss EPOCH 2: 0.0025
Valid Loss EPOCH 2: 0.0010
------------------------------


 30%|███       | 3/10 [00:01<00:02,  2.51it/s]

...

------------------------------
Train Loss EPOCH 8: 0.0004
Valid Loss EPOCH 8: 0.0003
------------------------------


 90%|█████████ | 9/10 [00:03<00:00,  2.52it/s]

------------------------------
Train Loss EPOCH 9: 0.0003
Valid Loss EPOCH 9: 0.0002
------------------------------


100%|██████████| 10/10 [00:03<00:00,  2.52it/s]


------------------------------
Train Loss EPOCH 10: 0.0002
Valid Loss EPOCH 10: 0.0001
------------------------------


In [39]:
import torch

def load_model(model_path, device='cpu'):
    model = UNet(in_channels=1, num_classes=1)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

def infer_single_image(model, image_path, device='cpu', target_size=(512, 512)):
    # Load and preprocess the image
    img = np.load(image_path)
    
    # If the image is complex, convert to real format
    if np.iscomplexobj(img):
        img = np.abs(img)
    
    img = Image.fromarray(img.astype(np.float32))
    img_resized = F.resize(img, target_size)
    img_tensor = F.to_tensor(img_resized).unsqueeze(0).to(device)  # Add batch dimension
    
    # Perform inference
    with torch.no_grad():
        prediction = model(img_tensor)
    
    # Convert the prediction to a numpy array and remove the batch dimension
    prediction_np = prediction.squeeze().cpu().numpy()
    
    return prediction_np

def infer_batch(model, data_loader, device='cpu'):
    predictions = []
    
    model.eval()
    with torch.no_grad():
        for img_batch in data_loader:
            img_batch = img_batch[0].to(device)  # Assume the dataloader returns (img, mask)
            
            # Perform inference
            preds = model(img_batch)
            
            # Convert the predictions to numpy arrays and append to the list
            for pred in preds:
                predictions.append(pred.squeeze().cpu().numpy())
    
    return predictions

In [22]:
# Load the model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = load_model(MODEL_SAVE_PATH, device=device)

predicted_CMB = infer_single_image(model, "data/train_data/ILC_MW_Map_R0000_0.npy", device=device)


In [23]:
import healpy as hp
import s2fft
def mw_alm_2_hp_alm(MW_alm, lmax):
    '''MW_alm: 2D array of shape (Lmax, 2*Lmax-1) (MW sampling, McEwen & Wiaux)
    '''
    # Initialize the 1D hp_alm array with the appropriate size
    hp_alm = np.zeros(hp.Alm.getsize(lmax), dtype=np.complex128)
        
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            index = hp.Alm.getidx(lmax, l, abs(m))
            if m < 0:
                hp_alm[index] = (-1)**m * np.conj(MW_alm[l, lmax + m])
            else:
                hp_alm[index] = MW_alm[l, lmax + m]

    return hp_alm

def hp_alm_2_mw_alm(hp_alm, L_max):
    MW_alm = np.zeros((L_max, 2 * L_max - 1), dtype=np.complex128)

    for l in range(L_max):
        for m in range(-l, l + 1):
            index = hp.Alm.getidx(L_max - 1, l, abs(m))
            if m < 0:
                MW_alm[l, L_max + m - 1] = (-1) ** m * np.conj(hp_alm[index])
            else:
                MW_alm[l, L_max + m - 1] = hp_alm[index]

    return MW_alm

def visualize_wavelet_coefficient_map(MW_Pix_Map, title, Frequency, min=None, max=None):
    """
    Processes a wavelet coefficient map and visualizes it using HEALPix mollview.
    Parameters:
        original_70: numpy array representing the wavelet coefficient map.
    Returns:
        Displays a mollview map.
    """
    # import healpy as hp  # Make sure to import healpy
    # from s2fft import forward  # Assuming s2fft.forward is imported
    if MW_Pix_Map.shape[0] != 1:
        L_max = MW_Pix_Map.shape[0]
    else:
        L_max = MW_Pix_Map.shape[1]
    original_map_alm = s2fft.forward(MW_Pix_Map, L=L_max)
    # print("Original map alm shape:", original_map_alm.shape)
    
    original_map_hp_alm = mw_alm_2_hp_alm(original_map_alm, L_max - 1)
    original_hp_map = hp.alm2map(original_map_hp_alm, nside=(L_max - 1)//2)

    hp.mollview(
        # original_hp_map * 1e5,
        original_hp_map,
        coord=["G"],
        title=title+Frequency,
        # unit=r"$1e5$K",
        # min=min, max=max,  # Uncomment and adjust these as necessary for better visualization contrast
    )


In [24]:
visualize_wavelet_coefficient_map(predicted_CMB, "Predicted CMB Map ", "100")

ValueError: could not broadcast input array from shape (1,512,512) into shape (1,512,1023)

In [44]:
# Example with a non-square input shape

input_shape = (256, 511)  # Height x Width

model = UNet(in_channels=1, num_classes=1)

# Simulate a batch of 1 input image with the shape 1x1x256x511 (batch_size, channels, height, width)
dummy_input = torch.randn(1, 1, input_shape[0], input_shape[1])

output = model(dummy_input)

print("Input shape:", dummy_input.shape)
print("Output shape:", output.shape)


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 33 but got size 32 for tensor number 1 in the list.